# Day10_2: A/B 테스트 실전 프로젝트 (A/B Test Practical Project)

## 📚 학습 목표

**Part 1: 프로젝트 기획 및 데이터 준비**
1. A/B 테스트 가설 설정 및 실험 설계하기
2. 표본 크기 및 실험 기간 계산하기
3. 다중 변수 A/B 테스트 데이터 탐색하기
4. 세그먼트별 데이터 전처리하기
5. 기초 통계량 및 그룹 간 비교하기

**Part 2: 분석 및 의사결정**
1. 통계적 검정 파이프라인 구축하기
2. Plotly 대시보드로 결과 시각화하기
3. 세그먼트별 심층 분석하기
4. 비즈니스 의사결정 권장사항 도출하기

---

## 🎯 왜 이것을 배우나요?

| 개념 | 실무 활용 | 예시 |
|------|----------|------|
| 실험 설계 | 신뢰할 수 있는 테스트 수행 | 표본 크기, 실험 기간 결정 |
| 전체 파이프라인 | End-to-End 분석 역량 | 가설→데이터→분석→보고 |
| 세그먼트 분석 | 고객 그룹별 인사이트 | 모바일 vs PC, 신규 vs 기존 |
| 대시보드 | 경영진 커뮤니케이션 | 의사결정 지원 시각화 |
| 권장사항 도출 | 비즈니스 임팩트 | 매출 증가, ROI 예측 |

**분석가 관점**: A/B 테스트는 데이터 분석가의 핵심 역량입니다. 이 프로젝트에서 실제 비즈니스 시나리오를 처음부터 끝까지 경험합니다!

---

# 🎯 프로젝트 개요

## 비즈니스 시나리오

**회사**: ShopMate (전자상거래 플랫폼)  
**목표**: 결제 페이지 전환율 향상  
**실험**: 새로운 결제 UI 디자인 테스트

### 실험 설계

| 항목 | 내용 |
|------|------|
| **Control (A)** | 기존 결제 페이지 - 2단계 결제 프로세스 |
| **Treatment (B)** | 신규 결제 페이지 - 1페이지 원클릭 결제 |
| **기간** | 2주 (2025-01-01 ~ 2025-01-14) |
| **대상** | 결제 페이지 도달 사용자 전체 |
| **할당 비율** | 50:50 무작위 배정 |

### 핵심 지표 (KPIs)

| 지표 | 설명 | 목표 |
|------|------|------|
| **Primary**: 전환율 | 결제 완료 / 결제 페이지 방문 | +5% 이상 개선 |
| **Secondary**: 객단가 | 주문당 평균 결제 금액 | 유지 또는 개선 |
| **Secondary**: 이탈률 | 결제 페이지에서 이탈한 비율 | -3% 이상 감소 |

### 세그먼트

- **디바이스**: Mobile vs Desktop
- **고객 유형**: 신규 vs 기존
- **결제 수단**: 카드 vs 간편결제

---

# Part 1: 프로젝트 기획 및 데이터 준비

---

## 1.1 라이브러리 및 데이터 로드

In [ ]:
# 필요한 라이브러리 임포트
import numpy as np
import pandas as pd
from scipy import stats
from datetime import datetime, timedelta
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# 랜덤 시드 설정 (재현성)
np.random.seed(42)

print("✅ 라이브러리 임포트 완료")

### 💡 실무 예시: A/B 테스트 데이터 생성

실제 프로젝트에서는 데이터 웨어하우스에서 SQL로 추출하지만, 이 프로젝트에서는 현실적인 시뮬레이션 데이터를 생성합니다.

In [ ]:
# A/B 테스트 데이터 생성 함수
def generate_ab_test_data(n_users=10000):
    """
    전자상거래 A/B 테스트 데이터 생성
    
    Parameters:
    - n_users: 총 사용자 수
    
    Returns:
    - DataFrame: A/B 테스트 데이터
    """
    
    # 기본 사용자 정보
    user_ids = [f'user_{str(i).zfill(5)}' for i in range(n_users)]
    
    # A/B 그룹 할당 (50:50)
    groups = np.random.choice(['control', 'treatment'], size=n_users, p=[0.5, 0.5])
    
    # 세그먼트: 디바이스
    devices = np.random.choice(['mobile', 'desktop'], size=n_users, p=[0.65, 0.35])
    
    # 세그먼트: 고객 유형
    customer_types = np.random.choice(['new', 'returning'], size=n_users, p=[0.4, 0.6])
    
    # 세그먼트: 결제 수단
    payment_methods = np.random.choice(['card', 'easy_pay'], size=n_users, p=[0.45, 0.55])
    
    # 날짜 (2주간)
    start_date = datetime(2025, 1, 1)
    dates = [start_date + timedelta(days=np.random.randint(0, 14)) for _ in range(n_users)]
    
    # 전환 여부 결정 (세그먼트별 다른 전환율 적용)
    conversions = []
    order_amounts = []
    page_time_seconds = []
    
    for i in range(n_users):
        # 기본 전환율 설정
        if groups[i] == 'control':
            base_rate = 0.12  # Control: 12% 기본 전환율
        else:
            base_rate = 0.145  # Treatment: 14.5% 기본 전환율 (+20.8% 개선)
        
        # 세그먼트별 전환율 조정
        # 모바일은 전환율이 낮음
        if devices[i] == 'mobile':
            base_rate *= 0.85
            # Treatment에서 모바일 개선 효과가 더 큼
            if groups[i] == 'treatment':
                base_rate *= 1.15
        
        # 기존 고객은 전환율이 높음
        if customer_types[i] == 'returning':
            base_rate *= 1.25
        
        # 간편결제는 전환율이 높음
        if payment_methods[i] == 'easy_pay':
            base_rate *= 1.1
        
        # 전환 여부
        converted = np.random.random() < base_rate
        conversions.append(1 if converted else 0)
        
        # 주문 금액 (전환된 경우만)
        if converted:
            # 평균 55,000원, 표준편차 25,000원
            amount = max(10000, np.random.normal(55000, 25000))
            # 기존 고객은 객단가가 높음
            if customer_types[i] == 'returning':
                amount *= 1.15
            order_amounts.append(round(amount, -2))  # 100원 단위 반올림
        else:
            order_amounts.append(0)
        
        # 페이지 체류 시간 (초)
        if groups[i] == 'control':
            time_spent = max(10, np.random.normal(120, 45))  # 기존: 평균 2분
        else:
            time_spent = max(10, np.random.normal(85, 35))  # 신규: 평균 1분 25초 (빠른 결제)
        page_time_seconds.append(round(time_spent))
    
    # DataFrame 생성
    df = pd.DataFrame({
        'user_id': user_ids,
        'date': dates,
        'group': groups,
        'device': devices,
        'customer_type': customer_types,
        'payment_method': payment_methods,
        'converted': conversions,
        'order_amount': order_amounts,
        'page_time_seconds': page_time_seconds
    })
    
    return df

# 데이터 생성
df = generate_ab_test_data(n_users=10000)

print(f"✅ A/B 테스트 데이터 생성 완료")
print(f"총 사용자 수: {len(df):,}명")
print(f"실험 기간: {df['date'].min().strftime('%Y-%m-%d')} ~ {df['date'].max().strftime('%Y-%m-%d')}")

---

## 1.2 데이터 탐색 (EDA)

In [ ]:
# 데이터 기본 정보 확인
print("데이터 형태:", df.shape)
print("\n컬럼 정보:")
print(df.dtypes)
print("\n처음 5행:")
df.head()

In [ ]:
# A/B 그룹별 기초 통계
print("=" * 60)
print("A/B 그룹별 분포")
print("=" * 60)

group_summary = df.groupby('group').agg({
    'user_id': 'count',
    'converted': ['sum', 'mean'],
    'order_amount': 'mean',
    'page_time_seconds': 'mean'
}).round(2)

group_summary.columns = ['사용자수', '전환수', '전환율', '평균주문금액', '평균체류시간(초)']
print(group_summary)

In [ ]:
# 세그먼트별 분포 확인
print("\n세그먼트별 분포")
print("-" * 40)

for col in ['device', 'customer_type', 'payment_method']:
    print(f"\n{col}:")
    segment_dist = df.groupby(['group', col]).size().unstack(fill_value=0)
    print(segment_dist)

### 💡 실무 예시: A/A 테스트 검증 (SRM 체크)

Sample Ratio Mismatch (SRM)를 확인하여 실험 설계가 올바른지 검증합니다.

In [ ]:
# SRM (Sample Ratio Mismatch) 검증
# 기대 비율: 50:50
observed_counts = df['group'].value_counts()
expected_counts = [len(df) / 2, len(df) / 2]

# Chi-square 검정
chi2, p_value = stats.chisquare(observed_counts.values, expected_counts)

print("SRM (Sample Ratio Mismatch) 검증")
print("=" * 50)
print(f"Control: {observed_counts['control']:,}명 ({observed_counts['control']/len(df)*100:.1f}%)")
print(f"Treatment: {observed_counts['treatment']:,}명 ({observed_counts['treatment']/len(df)*100:.1f}%)")
print(f"\nChi-square: {chi2:.4f}")
print(f"p-value: {p_value:.4f}")

if p_value > 0.05:
    print("\n✅ SRM 통과: 그룹 할당이 올바르게 되었습니다.")
else:
    print("\n⚠️ SRM 경고: 그룹 할당에 문제가 있을 수 있습니다!")

---

## 1.3 가설 설정

In [ ]:
# 가설 설정 문서화
hypotheses = pd.DataFrame({
    '지표': ['전환율', '객단가', '체류시간'],
    '귀무가설 (H₀)': [
        'Control과 Treatment의 전환율에 차이가 없다',
        'Control과 Treatment의 객단가에 차이가 없다',
        'Control과 Treatment의 체류시간에 차이가 없다'
    ],
    '대립가설 (H₁)': [
        'Treatment의 전환율이 더 높다 (단측)',
        'Treatment의 객단가가 다르다 (양측)',
        'Treatment의 체류시간이 더 짧다 (단측)'
    ],
    '유의수준 (α)': [0.05, 0.05, 0.05]
})

print("📋 가설 설정")
print("=" * 80)
print(hypotheses.to_string(index=False))

---

## 1.4 표본 크기 계산 (Power Analysis)

### 💡 실무 예시: 사전 표본 크기 계산

실험 전에 필요한 표본 크기를 계산하여 실험 기간을 결정합니다.

In [ ]:
# 표본 크기 계산 함수
def calculate_sample_size(baseline_rate, mde, alpha=0.05, power=0.80):
    """
    A/B 테스트에 필요한 표본 크기 계산
    
    Parameters:
    - baseline_rate: 기존 전환율 (예: 0.12 = 12%)
    - mde: Minimum Detectable Effect (최소 감지 효과, 예: 0.15 = 15% 상대적 개선)
    - alpha: 유의수준 (기본 0.05)
    - power: 검정력 (기본 0.80)
    
    Returns:
    - 그룹당 필요 표본 크기
    """
    # 목표 전환율
    target_rate = baseline_rate * (1 + mde)
    
    # 풀링된 표준편차
    pooled_p = (baseline_rate + target_rate) / 2
    pooled_se = np.sqrt(2 * pooled_p * (1 - pooled_p))
    
    # z값
    z_alpha = stats.norm.ppf(1 - alpha/2)
    z_beta = stats.norm.ppf(power)
    
    # 표본 크기 계산
    effect_size = abs(target_rate - baseline_rate)
    n = ((z_alpha + z_beta) ** 2 * pooled_se ** 2) / (effect_size ** 2)
    
    return int(np.ceil(n))

# 우리 실험의 표본 크기 계산
baseline = 0.12  # 기존 전환율 12%
mde = 0.15  # 15% 상대적 개선 목표

required_n = calculate_sample_size(baseline, mde)

print("📊 표본 크기 계산 (Power Analysis)")
print("=" * 50)
print(f"기존 전환율: {baseline*100:.1f}%")
print(f"목표 개선율: {mde*100:.1f}% (상대적)")
print(f"목표 전환율: {baseline*(1+mde)*100:.2f}%")
print(f"유의수준 (α): 0.05")
print(f"검정력 (1-β): 0.80")
print(f"\n그룹당 필요 표본 크기: {required_n:,}명")
print(f"총 필요 표본 크기: {required_n*2:,}명")
print(f"\n현재 데이터 크기: {len(df):,}명")

if len(df) >= required_n * 2:
    print("✅ 충분한 표본 크기 확보!")
else:
    print(f"⚠️ 추가로 {required_n*2 - len(df):,}명 필요")

---

# Part 2: 분석 및 의사결정

---

## 2.1 전환율 분석

In [ ]:
# 전환율 비율 검정 (z-test)
def conversion_rate_test(df, group_col='group', conv_col='converted'):
    """
    A/B 그룹 간 전환율 차이 검정
    
    Returns:
    - 결과 딕셔너리
    """
    # 그룹별 통계
    control = df[df[group_col] == 'control'][conv_col]
    treatment = df[df[group_col] == 'treatment'][conv_col]
    
    n_c, n_t = len(control), len(treatment)
    conv_c, conv_t = control.sum(), treatment.sum()
    rate_c, rate_t = conv_c / n_c, conv_t / n_t
    
    # 풀링된 비율
    pooled_rate = (conv_c + conv_t) / (n_c + n_t)
    pooled_se = np.sqrt(pooled_rate * (1 - pooled_rate) * (1/n_c + 1/n_t))
    
    # z-statistic
    z_stat = (rate_t - rate_c) / pooled_se
    
    # p-value (단측: Treatment > Control)
    p_value_one_sided = 1 - stats.norm.cdf(z_stat)
    p_value_two_sided = 2 * (1 - stats.norm.cdf(abs(z_stat)))
    
    # 상대적 개선율
    relative_lift = (rate_t - rate_c) / rate_c * 100
    
    # 95% 신뢰구간
    se_diff = np.sqrt(rate_c*(1-rate_c)/n_c + rate_t*(1-rate_t)/n_t)
    ci_lower = (rate_t - rate_c) - 1.96 * se_diff
    ci_upper = (rate_t - rate_c) + 1.96 * se_diff
    
    return {
        'control_n': n_c,
        'treatment_n': n_t,
        'control_conversions': conv_c,
        'treatment_conversions': conv_t,
        'control_rate': rate_c,
        'treatment_rate': rate_t,
        'absolute_lift': rate_t - rate_c,
        'relative_lift': relative_lift,
        'z_statistic': z_stat,
        'p_value_one_sided': p_value_one_sided,
        'p_value_two_sided': p_value_two_sided,
        'ci_lower': ci_lower,
        'ci_upper': ci_upper,
        'significant': p_value_one_sided < 0.05
    }

# 전환율 검정 실행
conversion_results = conversion_rate_test(df)

print("📊 전환율 분석 결과")
print("=" * 60)
print(f"\nControl (기존 UI):")
print(f"  사용자: {conversion_results['control_n']:,}명")
print(f"  전환: {conversion_results['control_conversions']:,}건")
print(f"  전환율: {conversion_results['control_rate']*100:.2f}%")

print(f"\nTreatment (신규 UI):")
print(f"  사용자: {conversion_results['treatment_n']:,}명")
print(f"  전환: {conversion_results['treatment_conversions']:,}건")
print(f"  전환율: {conversion_results['treatment_rate']*100:.2f}%")

print(f"\n개선 효과:")
print(f"  절대 개선: {conversion_results['absolute_lift']*100:+.2f}%p")
print(f"  상대 개선: {conversion_results['relative_lift']:+.1f}%")
print(f"  95% 신뢰구간: [{conversion_results['ci_lower']*100:.2f}%, {conversion_results['ci_upper']*100:.2f}%]")

print(f"\n통계적 검정:")
print(f"  z-statistic: {conversion_results['z_statistic']:.4f}")
print(f"  p-value (단측): {conversion_results['p_value_one_sided']:.4f}")

print("\n" + "=" * 60)
if conversion_results['significant']:
    print("✅ 결론: 신규 UI가 전환율을 통계적으로 유의하게 개선했습니다!")
else:
    print("❌ 결론: 통계적으로 유의한 차이를 발견하지 못했습니다.")

---

## 2.2 객단가 분석

In [ ]:
# 전환된 사용자만 필터링
df_converted = df[df['converted'] == 1].copy()

# 그룹별 객단가
control_amounts = df_converted[df_converted['group'] == 'control']['order_amount']
treatment_amounts = df_converted[df_converted['group'] == 'treatment']['order_amount']

# t-test (양측)
t_stat, p_value = stats.ttest_ind(control_amounts, treatment_amounts)

print("📊 객단가 분석 결과")
print("=" * 60)
print(f"\nControl:")
print(f"  전환 고객: {len(control_amounts):,}명")
print(f"  평균 객단가: {control_amounts.mean():,.0f}원")
print(f"  표준편차: {control_amounts.std():,.0f}원")

print(f"\nTreatment:")
print(f"  전환 고객: {len(treatment_amounts):,}명")
print(f"  평균 객단가: {treatment_amounts.mean():,.0f}원")
print(f"  표준편차: {treatment_amounts.std():,.0f}원")

diff = treatment_amounts.mean() - control_amounts.mean()
print(f"\n차이: {diff:+,.0f}원 ({diff/control_amounts.mean()*100:+.1f}%)")
print(f"\nt-statistic: {t_stat:.4f}")
print(f"p-value: {p_value:.4f}")

if p_value < 0.05:
    print("\n✅ 객단가에 통계적으로 유의한 차이가 있습니다.")
else:
    print("\n✅ 객단가는 유지되었습니다 (유의한 차이 없음).")

---

## 2.3 체류시간 분석

In [ ]:
# 그룹별 체류시간
control_time = df[df['group'] == 'control']['page_time_seconds']
treatment_time = df[df['group'] == 'treatment']['page_time_seconds']

# t-test
t_stat, p_value = stats.ttest_ind(control_time, treatment_time)

print("📊 체류시간 분석 결과")
print("=" * 60)
print(f"\nControl:")
print(f"  평균 체류시간: {control_time.mean():.1f}초 ({control_time.mean()/60:.1f}분)")
print(f"  표준편차: {control_time.std():.1f}초")

print(f"\nTreatment:")
print(f"  평균 체류시간: {treatment_time.mean():.1f}초 ({treatment_time.mean()/60:.1f}분)")
print(f"  표준편차: {treatment_time.std():.1f}초")

diff = treatment_time.mean() - control_time.mean()
print(f"\n차이: {diff:+.1f}초 ({diff/control_time.mean()*100:+.1f}%)")
print(f"\nt-statistic: {t_stat:.4f}")
print(f"p-value: {p_value:.4f}")

if p_value < 0.05 and diff < 0:
    print("\n✅ 결제 프로세스가 유의하게 빨라졌습니다!")
else:
    print("\n❌ 체류시간에 유의한 변화가 없습니다.")

---

## 2.4 세그먼트별 분석

### 💡 실무 예시: 세그먼트별 효과 차이 분석

모든 사용자에게 동일한 효과가 나타나지 않을 수 있습니다. 세그먼트별로 분석하면 더 깊은 인사이트를 얻을 수 있습니다.

In [ ]:
# 세그먼트별 분석 함수
def segment_analysis(df, segment_col):
    """
    세그먼트별 A/B 테스트 분석
    
    Returns:
    - DataFrame: 세그먼트별 결과
    """
    results = []
    
    for segment in df[segment_col].unique():
        df_seg = df[df[segment_col] == segment]
        
        # 그룹별 전환율
        control = df_seg[df_seg['group'] == 'control']
        treatment = df_seg[df_seg['group'] == 'treatment']
        
        rate_c = control['converted'].mean()
        rate_t = treatment['converted'].mean()
        
        # 검정
        test_result = conversion_rate_test(df_seg)
        
        results.append({
            'segment': segment,
            'control_n': len(control),
            'treatment_n': len(treatment),
            'control_rate': rate_c * 100,
            'treatment_rate': rate_t * 100,
            'relative_lift': test_result['relative_lift'],
            'p_value': test_result['p_value_one_sided'],
            'significant': test_result['significant']
        })
    
    return pd.DataFrame(results)

# 디바이스별 분석
print("📱 디바이스별 분석")
print("=" * 80)
device_results = segment_analysis(df, 'device')
print(device_results.to_string(index=False))

In [ ]:
# 고객 유형별 분석
print("\n👥 고객 유형별 분석")
print("=" * 80)
customer_results = segment_analysis(df, 'customer_type')
print(customer_results.to_string(index=False))

In [ ]:
# 결제 수단별 분석
print("\n💳 결제 수단별 분석")
print("=" * 80)
payment_results = segment_analysis(df, 'payment_method')
print(payment_results.to_string(index=False))

---

## 2.5 Plotly 대시보드 시각화

### 💡 실무 예시: 경영진 보고용 대시보드

In [ ]:
# 전환율 비교 차트
fig1 = go.Figure()

# Control
fig1.add_trace(go.Bar(
    name='Control (기존 UI)',
    x=['전체'],
    y=[conversion_results['control_rate'] * 100],
    text=[f"{conversion_results['control_rate']*100:.2f}%"],
    textposition='outside',
    marker_color='#636EFA'
))

# Treatment
fig1.add_trace(go.Bar(
    name='Treatment (신규 UI)',
    x=['전체'],
    y=[conversion_results['treatment_rate'] * 100],
    text=[f"{conversion_results['treatment_rate']*100:.2f}%"],
    textposition='outside',
    marker_color='#EF553B'
))

fig1.update_layout(
    title=dict(text='A/B 테스트: 전환율 비교', font=dict(size=18)),
    yaxis_title='전환율 (%)',
    barmode='group',
    height=400,
    showlegend=True
)

# p-value 주석 추가
significance_text = "p < 0.05 ✅" if conversion_results['significant'] else "p >= 0.05"
fig1.add_annotation(
    x=0.5, y=max(conversion_results['control_rate'], conversion_results['treatment_rate']) * 100 + 2,
    text=f"상대 개선: {conversion_results['relative_lift']:+.1f}% | {significance_text}",
    showarrow=False,
    font=dict(size=14, color='green' if conversion_results['significant'] else 'gray')
)

fig1.show()

In [ ]:
# 일별 전환율 트렌드
daily_conversion = df.groupby(['date', 'group']).agg({
    'converted': 'mean'
}).reset_index()
daily_conversion['conversion_rate'] = daily_conversion['converted'] * 100

fig2 = px.line(
    daily_conversion,
    x='date',
    y='conversion_rate',
    color='group',
    title='일별 전환율 트렌드',
    labels={'date': '날짜', 'conversion_rate': '전환율 (%)', 'group': '그룹'},
    color_discrete_map={'control': '#636EFA', 'treatment': '#EF553B'},
    markers=True
)

fig2.update_layout(height=400)
fig2.show()

In [ ]:
# 세그먼트별 전환율 비교 (히트맵 스타일)
segment_pivot = df.pivot_table(
    values='converted',
    index=['device', 'customer_type'],
    columns='group',
    aggfunc='mean'
) * 100

segment_pivot['lift'] = segment_pivot['treatment'] - segment_pivot['control']
segment_pivot = segment_pivot.reset_index()

# 막대 그래프로 세그먼트별 개선율 시각화
segment_pivot['segment'] = segment_pivot['device'] + ' / ' + segment_pivot['customer_type']

fig3 = go.Figure()

fig3.add_trace(go.Bar(
    x=segment_pivot['segment'],
    y=segment_pivot['control'],
    name='Control',
    marker_color='#636EFA'
))

fig3.add_trace(go.Bar(
    x=segment_pivot['segment'],
    y=segment_pivot['treatment'],
    name='Treatment',
    marker_color='#EF553B'
))

fig3.update_layout(
    title='세그먼트별 전환율 비교 (디바이스 x 고객유형)',
    xaxis_title='세그먼트',
    yaxis_title='전환율 (%)',
    barmode='group',
    height=450
)

fig3.show()

In [ ]:
# 종합 대시보드 (Subplots)
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=(
        '전환율 비교',
        '객단가 분포',
        '일별 전환율 트렌드',
        '디바이스별 개선율'
    ),
    specs=[
        [{"type": "bar"}, {"type": "box"}],
        [{"type": "scatter"}, {"type": "bar"}]
    ]
)

# 1. 전환율 비교
fig.add_trace(
    go.Bar(
        x=['Control', 'Treatment'],
        y=[conversion_results['control_rate']*100, conversion_results['treatment_rate']*100],
        marker_color=['#636EFA', '#EF553B'],
        text=[f"{conversion_results['control_rate']*100:.2f}%", f"{conversion_results['treatment_rate']*100:.2f}%"],
        textposition='outside'
    ),
    row=1, col=1
)

# 2. 객단가 분포
fig.add_trace(
    go.Box(y=control_amounts, name='Control', marker_color='#636EFA'),
    row=1, col=2
)
fig.add_trace(
    go.Box(y=treatment_amounts, name='Treatment', marker_color='#EF553B'),
    row=1, col=2
)

# 3. 일별 트렌드
for group in ['control', 'treatment']:
    data = daily_conversion[daily_conversion['group'] == group]
    color = '#636EFA' if group == 'control' else '#EF553B'
    fig.add_trace(
        go.Scatter(
            x=data['date'],
            y=data['conversion_rate'],
            name=group.capitalize(),
            mode='lines+markers',
            line=dict(color=color)
        ),
        row=2, col=1
    )

# 4. 디바이스별 개선율
fig.add_trace(
    go.Bar(
        x=device_results['segment'],
        y=device_results['relative_lift'],
        marker_color=['#00CC96' if x > 0 else '#EF553B' for x in device_results['relative_lift']],
        text=[f"{x:+.1f}%" for x in device_results['relative_lift']],
        textposition='outside'
    ),
    row=2, col=2
)

fig.update_layout(
    height=700,
    title_text='A/B 테스트 종합 대시보드',
    showlegend=False
)

fig.show()

---

## 2.6 비즈니스 임팩트 계산

In [ ]:
# 비즈니스 임팩트 계산
def calculate_business_impact(results, monthly_visitors=100000, avg_order_value=55000):
    """
    A/B 테스트 결과를 비즈니스 임팩트로 환산
    
    Parameters:
    - results: conversion_rate_test 결과
    - monthly_visitors: 월간 결제 페이지 방문자
    - avg_order_value: 평균 주문 금액
    """
    # 현재 매출 (Control 기준)
    current_conversions = monthly_visitors * results['control_rate']
    current_revenue = current_conversions * avg_order_value
    
    # 예상 매출 (Treatment 적용 시)
    expected_conversions = monthly_visitors * results['treatment_rate']
    expected_revenue = expected_conversions * avg_order_value
    
    # 증가분
    additional_conversions = expected_conversions - current_conversions
    additional_revenue_monthly = expected_revenue - current_revenue
    additional_revenue_yearly = additional_revenue_monthly * 12
    
    return {
        'monthly_visitors': monthly_visitors,
        'current_conversions': current_conversions,
        'expected_conversions': expected_conversions,
        'additional_conversions_monthly': additional_conversions,
        'current_revenue_monthly': current_revenue,
        'expected_revenue_monthly': expected_revenue,
        'additional_revenue_monthly': additional_revenue_monthly,
        'additional_revenue_yearly': additional_revenue_yearly
    }

# 임팩트 계산
impact = calculate_business_impact(conversion_results)

print("💰 비즈니스 임팩트 분석")
print("=" * 60)
print(f"\n가정:")
print(f"  월간 결제 페이지 방문자: {impact['monthly_visitors']:,}명")
print(f"  평균 주문 금액: 55,000원")

print(f"\n현재 (Control):")
print(f"  월간 전환: {impact['current_conversions']:,.0f}건")
print(f"  월간 매출: {impact['current_revenue_monthly']/1e8:.2f}억원")

print(f"\n예상 (Treatment 전체 적용 시):")
print(f"  월간 전환: {impact['expected_conversions']:,.0f}건")
print(f"  월간 매출: {impact['expected_revenue_monthly']/1e8:.2f}억원")

print(f"\n예상 증가분:")
print(f"  월간 추가 전환: +{impact['additional_conversions_monthly']:,.0f}건")
print(f"  월간 추가 매출: +{impact['additional_revenue_monthly']/1e8:.2f}억원")
print(f"  연간 추가 매출: +{impact['additional_revenue_yearly']/1e8:.2f}억원")

print("\n" + "=" * 60)

---

## 2.7 최종 보고서 및 권장사항

In [ ]:
# 최종 보고서 생성
print("="*80)
print("📋 A/B 테스트 최종 보고서")
print("="*80)

print("\n1. 실험 개요")
print("-"*40)
print(f"   실험명: 결제 페이지 UI 개선 테스트")
print(f"   기간: 2025-01-01 ~ 2025-01-14 (2주)")
print(f"   총 참여자: {len(df):,}명")
print(f"   할당: Control {conversion_results['control_n']:,}명 / Treatment {conversion_results['treatment_n']:,}명")

print("\n2. 핵심 결과")
print("-"*40)
print(f"   전환율: {conversion_results['control_rate']*100:.2f}% -> {conversion_results['treatment_rate']*100:.2f}%")
print(f"   개선율: {conversion_results['relative_lift']:+.1f}%")
print(f"   통계적 유의성: {'유의함 (p<0.05)' if conversion_results['significant'] else '유의하지 않음'}")
print(f"   객단가: 유의한 변화 없음 (유지됨)")
print(f"   체류시간: {(treatment_time.mean() - control_time.mean()):.1f}초 단축")

print("\n3. 세그먼트별 인사이트")
print("-"*40)
print(f"   모바일: 가장 큰 개선 효과 (신규 UI 최적화 효과)")
print(f"   신규 고객: 기존 고객 대비 개선율 낮음 (신뢰 구축 필요)")
print(f"   간편결제: 가장 높은 전환율 (결제 편의성 시너지)")

print("\n4. 비즈니스 임팩트")
print("-"*40)
print(f"   예상 월간 추가 매출: +{impact['additional_revenue_monthly']/1e8:.2f}억원")
print(f"   예상 연간 추가 매출: +{impact['additional_revenue_yearly']/1e8:.2f}억원")

print("\n5. 권장사항")
print("-"*40)
if conversion_results['significant']:
    print("   ✅ [권장] 신규 UI를 전체 사용자에게 적용")
    print("   ✅ [권장] 모바일 사용자 타겟 마케팅 강화")
    print("   ✅ [권장] 간편결제 프로모션 확대")
    print("   ⚠️  [모니터링] 신규 고객 세그먼트 추가 분석 필요")
    print("   ⚠️  [모니터링] 배포 후 2주간 지표 모니터링 권장")
else:
    print("   ❌ 현재 결과로는 전체 배포 권장하지 않음")
    print("   ⚠️  추가 테스트 또는 UI 개선 필요")

print("\n6. 후속 실험 제안")
print("-"*40)
print("   1) 신규 고객 전용 가이드 UI 테스트")
print("   2) 결제 수단별 최적화 UI 테스트")
print("   3) 프로그레스바 추가 효과 테스트")

print("\n" + "="*80)
print(f"보고서 작성일: {datetime.now().strftime('%Y-%m-%d %H:%M')}")
print("="*80)

---

## 🎯 실습 퀴즈

**난이도**: ⭐ (쉬움) ~ ⭐⭐⭐⭐⭐ (어려움)

---

### Q1. 표본 크기 계산 ⭐

**문제**: 기존 전환율 8%, 목표 개선율 20% (상대적)일 때, 그룹당 필요한 표본 크기를 계산하세요.

```python
baseline_rate = 0.08
mde = 0.20  # 20% 상대적 개선
```

**기대 결과**: 그룹당 필요 표본 크기 (정수)

In [ ]:
baseline_rate = 0.08
mde = 0.20
# 여기에 코드를 작성하세요


### Q2. SRM 검증 ⭐⭐

**문제**: 다음 그룹 분포가 50:50 할당에서 유의한 편차가 있는지 Chi-square 검정으로 확인하세요.

```python
control_count = 4850
treatment_count = 5150
```

**기대 결과**: Chi-square 값, p-value, SRM 통과 여부

In [ ]:
control_count = 4850
treatment_count = 5150
# 여기에 코드를 작성하세요


### Q3. 전환율 z-검정 ⭐⭐

**문제**: 두 그룹의 전환율 차이가 통계적으로 유의한지 z-검정을 수행하세요.

```python
control_n = 5000
control_conversions = 600
treatment_n = 5000
treatment_conversions = 720
```

**기대 결과**: Control 전환율, Treatment 전환율, z-statistic, p-value, 결론

In [ ]:
control_n = 5000
control_conversions = 600
treatment_n = 5000
treatment_conversions = 720
# 여기에 코드를 작성하세요


### Q4. 세그먼트별 전환율 계산 ⭐⭐⭐

**문제**: 우리 실습 데이터(`df`)에서 디바이스와 고객 유형별 조합의 전환율을 피벗 테이블로 계산하세요.

**기대 결과**: 2x2 피벗 테이블 (행: device, 열: customer_type, 값: 전환율 %)

In [ ]:
# 여기에 코드를 작성하세요


### Q5. 95% 신뢰구간 계산 ⭐⭐⭐

**문제**: Treatment 그룹 전환율의 95% 신뢰구간을 계산하세요.

```python
n = 5000
conversions = 720
```

**힌트**: 신뢰구간 = p +- z * sqrt(p*(1-p)/n), z=1.96

**기대 결과**: 전환율과 95% 신뢰구간 [하한, 상한]

In [ ]:
n = 5000
conversions = 720
# 여기에 코드를 작성하세요


### Q6. 비즈니스 임팩트 계산 ⭐⭐⭐

**문제**: 전환율이 10%에서 12%로 개선되었을 때, 월간 추가 매출을 계산하세요.

```python
monthly_visitors = 50000
avg_order_value = 45000  # 원
old_rate = 0.10
new_rate = 0.12
```

**기대 결과**: 추가 전환 수, 추가 매출 (원)

In [ ]:
monthly_visitors = 50000
avg_order_value = 45000
old_rate = 0.10
new_rate = 0.12
# 여기에 코드를 작성하세요


### Q7. Plotly 막대 그래프 ⭐⭐⭐⭐

**문제**: 세 개 버전(A, B, C)의 전환율을 비교하는 그룹 막대 그래프를 Plotly로 그리세요.

```python
versions = ['Version A', 'Version B', 'Version C']
conversion_rates = [8.5, 10.2, 9.8]  # %
```

**요구사항**: 막대 위에 값 표시, 제목 포함

In [ ]:
versions = ['Version A', 'Version B', 'Version C']
conversion_rates = [8.5, 10.2, 9.8]
# 여기에 코드를 작성하세요


### Q8. 일별 트렌드 시각화 ⭐⭐⭐⭐

**문제**: 우리 데이터(`df`)에서 그룹별 일별 평균 객단가 트렌드를 선 그래프로 시각화하세요.

**요구사항**: 
- 전환된 사용자만 대상
- 그룹별 색상 구분
- 마커 표시

In [ ]:
# 여기에 코드를 작성하세요


### Q9. 종합 분석 함수 ⭐⭐⭐⭐⭐

**문제**: A/B 테스트 데이터를 받아 전환율 검정, 개선율, 통계적 유의성을 한 번에 분석하는 함수를 작성하세요.

```python
def analyze_ab_test(control_conv, control_n, treatment_conv, treatment_n, alpha=0.05):
    """
    A/B 테스트 종합 분석
    
    Returns:
    - dict: 분석 결과 (전환율, 개선율, z값, p값, 유의성)
    """
    pass
```

**테스트 데이터**:
- Control: 600/5000 전환
- Treatment: 720/5000 전환

In [ ]:
# 여기에 코드를 작성하세요


### Q10. 전체 파이프라인 프로젝트 ⭐⭐⭐⭐⭐

**문제**: 다음 시나리오에 대해 A/B 테스트 전체 파이프라인을 구현하세요.

**시나리오**: 이메일 마케팅 제목 A/B 테스트
- Control: 기존 제목 (오픈율 15%)
- Treatment: 이모지 포함 제목 (오픈율 18%)
- 각 그룹 3000명

**수행 작업**:
1. 가상 데이터 생성 (np.random.binomial 사용)
2. 기초 통계량 계산
3. 통계적 검정 수행
4. Plotly로 결과 시각화
5. 비즈니스 권장사항 출력

In [ ]:
# 여기에 코드를 작성하세요


---

## 📊 학습 정리

### Part 1: 프로젝트 기획 핵심 요약

| 단계 | 핵심 활동 | 주요 산출물 |
|------|----------|------------|
| 가설 설정 | H0/H1 정의, KPI 선정 | 가설 문서 |
| 실험 설계 | 표본 크기 계산, 기간 결정 | 실험 설계서 |
| 데이터 수집 | 그룹 할당, 지표 트래킹 | 원시 데이터 |
| SRM 검증 | 그룹 비율 검증 | Chi-square 결과 |
| 기초 통계 | 그룹별 요약 통계 | 기술 통계표 |

### Part 2: 분석 및 의사결정 핵심 요약

| 분석 유형 | 검정 방법 | Python 함수 | 활용 상황 |
|----------|----------|-------------|----------|
| 전환율 비교 | z-test (비율) | 직접 구현 | 클릭, 구매 전환 |
| 평균 비교 | t-test | stats.ttest_ind() | 객단가, 체류시간 |
| 범주형 비교 | Chi-square | stats.chisquare() | SRM, 세그먼트 분포 |
| 세그먼트 분석 | 그룹별 검정 | 루프 적용 | 디바이스, 고객 유형 |

### 💡 실무 팁

1. **SRM 검증 필수**: 실험 분석 전 그룹 할당 검증으로 데이터 무결성 확인
2. **표본 크기 사전 계산**: Power Analysis로 실험 기간과 신뢰도 사전 확보
3. **세그먼트별 분석**: 전체 효과가 작아도 특정 세그먼트에서 큰 효과 가능
4. **신뢰구간 보고**: p-value만 아니라 효과 크기의 불확실성도 함께 보고
5. **비즈니스 임팩트 환산**: 통계적 유의성 + 매출 영향으로 의사결정 지원
6. **후속 실험 제안**: 한 번의 실험으로 끝이 아닌 지속적 개선 사이클 구축

### 📈 A/B 테스트 프로세스 플로우

```
[가설 설정] → [실험 설계] → [데이터 수집] → [SRM 검증]
      ↓
[기초 통계] → [통계적 검정] → [세그먼트 분석]
      ↓
[시각화] → [임팩트 계산] → [권장사항 도출] → [후속 실험]
```

### 🎯 핵심 공식

**전환율 z-검정**:
```
z = (p_t - p_c) / sqrt(p_pool * (1-p_pool) * (1/n_c + 1/n_t))
```

**95% 신뢰구간**:
```
CI = (p_t - p_c) +- 1.96 * sqrt(p_c*(1-p_c)/n_c + p_t*(1-p_t)/n_t)
```

**상대적 개선율 (Lift)**:
```
Lift = (p_t - p_c) / p_c * 100%
```